In [ ]:
from pyspark.sql import SparkSession,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
import unicodedata
import re
import utils as utils

In [2]:
source_db = "landing"
source_table = "aena"
db_name = "trusted"
table_name = "f_flujo_aereo"

In [3]:
spark = utils.create_context()

In [4]:
print(spark.conf.get("spark.sql.catalog.spark_catalog.warehouse"))

../data/warehouse


In [5]:
test = utils.read_iceberg_table(spark,"local_db","aena_barcelona")

test.show()

+----+---+--------------------+---------------------+
| AÑO|MES|     AEROPUERTO_AENA|PASAJEROS_POR_DESTINO|
+----+---+--------------------+---------------------+
|2004|  1|JT Barcelona-El Prat|              1487280|
|2004|  2|JT Barcelona-El Prat|              1681762|
|2004|  3|JT Barcelona-El Prat|              1943771|
|2004|  4|JT Barcelona-El Prat|              2030420|
|2004|  5|JT Barcelona-El Prat|              2128796|
|2004|  6|JT Barcelona-El Prat|              2203999|
|2004|  7|JT Barcelona-El Prat|              2377152|
|2004|  8|JT Barcelona-El Prat|              2503196|
|2004|  9|JT Barcelona-El Prat|              2262870|
|2004| 10|JT Barcelona-El Prat|              2186597|
|2004| 11|JT Barcelona-El Prat|              1846926|
|2004| 12|JT Barcelona-El Prat|              1711314|
|2005|  1|JT Barcelona-El Prat|              1619040|
|2005|  2|JT Barcelona-El Prat|              1766193|
|2005|  3|JT Barcelona-El Prat|              2159888|
|2005|  4|JT Barcelona-El Pr

In [ ]:
@udf(StringType())
def normalize_text(text:str):

    if test is None:
        return ""
    
    # 1. Normalizar (separar letras de tildes)
    texto_normalizado = unicodedata.normalize('NFKD', text)
    
    # 2. Eliminar acentos (caracteres "combining")
    texto_sin_acentos = ''.join(
        c for c in texto_normalizado if not unicodedata.combining(c)
    )
    
    # 3. Eliminar puntuación (todo lo que no sea letra, número o espacio)
    texto_sin_puntuacion = re.sub(r'[^A-Za-z0-9\s]', '', texto_sin_acentos)
    
    # 4. Eliminar espacios extra
    texto_limpio = ' '.join(texto_sin_puntuacion.split())
    
    # 5. Convertir a mayúsculas
    return texto_limpio.upper()

In [ ]:

df = utils.read_iceberg_table(spark,db_name,table_name)

# convert each column to normalized type
df = df.select(
    col("AÑO").alias("year"),
    col("MES").alias("month"),
    normalize_text(col("AEROPUERTO_AENA")).alias("aeirport"),
    col("PASAJEROS_POR_DESTINO").alias("numPassangers").cast(IntegerType())
)